Cellule 1 : Imports et connexions aux deux bases


In [8]:
import os
from dotenv import load_dotenv
import pyodbc
from datetime import datetime
import pandas as pd

# Charger les variables d'environnement
load_dotenv()

print("🔌 CONNEXION AUX BASES DE DONNÉES")
print("=" * 70)

# ========================================
# Configuration SOURCE (carter_cash)
# ========================================
server_source = os.getenv('DB_SERVER_SOURCE')
database_source = os.getenv('DB_DATABASE_SOURCE')
username_source = os.getenv('DB_USERNAME_SOURCE')
password_source = os.getenv('DB_PASSWORD_SOURCE')

driver = '{ODBC Driver 17 for SQL Server}'

# Connexion à la base SOURCE
cnxn_source = pyodbc.connect(
    'DRIVER=' + driver + 
    ';SERVER=' + server_source + 
    ';PORT=1433;DATABASE=' + database_source + 
    ';UID=' + username_source + 
    ';PWD=' + password_source
)
cursor_source = cnxn_source.cursor()

print("✅ Connexion réussie à la base SOURCE (carter_cash)")
print(f"   📦 Serveur : {server_source}")
print(f"   🗄️  Base : {database_source}")

# ========================================
# Configuration DWH (DWH_E5_projet_AUTO)
# ========================================
server_dwh = os.getenv('DB_SERVER_DWH')
database_dwh = os.getenv('DB_DATABASE_DWH')
username_dwh = os.getenv('DB_USERNAME_DWH')
password_dwh = os.getenv('DB_PASSWORD_DWH')

# Connexion à la base DWH
cnxn_dwh = pyodbc.connect(
    'DRIVER=' + driver + 
    ';SERVER=' + server_dwh + 
    ';PORT=1433;DATABASE=' + database_dwh + 
    ';UID=' + username_dwh + 
    ';PWD=' + password_dwh
)
cursor_dwh = cnxn_dwh.cursor()

print("\n✅ Connexion réussie à la base DWH (DWH_E5_projet_AUTO)")
print(f"   📦 Serveur : {server_dwh}")
print(f"   🗄️  Base : {database_dwh}")
print("=" * 70)

🔌 CONNEXION AUX BASES DE DONNÉES
✅ Connexion réussie à la base SOURCE (carter_cash)
   📦 Serveur : carter-cash-serveur.database.windows.net
   🗄️  Base : carter_cash

✅ Connexion réussie à la base DWH (DWH_E5_projet_AUTO)
   📦 Serveur : carter-cash-serveur.database.windows.net
   🗄️  Base : DWH_E5_projet_AUTO


Cellule 2 : Vérification des tables sources


In [9]:
print("\n🔍 VÉRIFICATION DES TABLES SOURCES")
print("=" * 70)

# Liste des tables sources à vérifier
tables_sources = [
    'USER_API',
    'DimensionsParModel',
    'PRODUIT',
    'CARACTERISTIQUES',
    'DIMENSIONS'
]

for table in tables_sources:
    try:
        cursor_source.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor_source.fetchone()[0]
        print(f"✅ {table} : {count} enregistrements")
    except Exception as e:
        print(f"❌ Erreur avec la table {table} : {e}")

print("=" * 70)


🔍 VÉRIFICATION DES TABLES SOURCES
✅ USER_API : 4 enregistrements
✅ DimensionsParModel : 191112 enregistrements
✅ PRODUIT : 11442 enregistrements
✅ CARACTERISTIQUES : 11442 enregistrements
✅ DIMENSIONS : 11180 enregistrements


Cellule 3 : ETL - Chargement de DIM_USER_API


In [10]:
print("\n📥 ETL - CHARGEMENT DE DIM_USER_API")
print("=" * 70)

# Extraction depuis la source
print("⏳ Extraction des données USER_API depuis carter_cash...")
cursor_source.execute("""
SELECT 
    ID_USER_API,
    username,
    email,
    full_name,
    Date_Création,
    Date_Derniere_Connexion
FROM USER_API
""")

users_source = cursor_source.fetchall()
print(f"✅ {len(users_source)} utilisateurs extraits")

# Transformation et Chargement avec SCD Type 2
print("\n⏳ Chargement dans DIM_USER_API avec gestion SCD Type 2...")

insert_count = 0
update_count = 0
skip_count = 0

for user in users_source:
    id_source = user[0]
    username = user[1]
    email = user[2]
    full_name = user[3]
    date_creation = user[4]
    date_derniere_connexion = user[5]
    
    # Vérifier si l'utilisateur existe déjà (version actuelle)
    cursor_dwh.execute("""
    SELECT SK_User_API, Username, Email, Full_Name, Date_Derniere_Connexion
    FROM DIM_USER_API
    WHERE ID_USER_API_Source = ? AND Flag_Actuel = 1
    """, id_source)
    
    existing_user = cursor_dwh.fetchone()
    
    if existing_user is None:
        # Nouvel utilisateur : insertion
        cursor_dwh.execute("""
        INSERT INTO DIM_USER_API (
            ID_USER_API_Source,
            Username,
            Email,
            Full_Name,
            Date_Creation,
            Date_Derniere_Connexion,
            Date_Debut_Validite,
            Date_Fin_Validite,
            Flag_Actuel
        )
        VALUES (?, ?, ?, ?, ?, ?, GETDATE(), NULL, 1)
        """, id_source, username, email, full_name, date_creation, date_derniere_connexion)
        insert_count += 1
        
    else:
        # Vérifier si les données ont changé (SCD Type 2)
        sk_user = existing_user[0]
        old_username = existing_user[1]
        old_email = existing_user[2]
        old_full_name = existing_user[3]
        old_last_login = existing_user[4]
        
        # Comparaison des changements significatifs
        has_changed = False
        
        # Comparaison sécurisée avec gestion des NULL
        if username != old_username:
            has_changed = True
        elif email != old_email:
            has_changed = True
        elif full_name != old_full_name:
            has_changed = True
        elif date_derniere_connexion != old_last_login:
            has_changed = True
        
        if has_changed:
            # Fermer l'ancienne version
            cursor_dwh.execute("""
            UPDATE DIM_USER_API
            SET Date_Fin_Validite = GETDATE(), Flag_Actuel = 0
            WHERE SK_User_API = ?
            """, sk_user)
            
            # Insérer la nouvelle version
            cursor_dwh.execute("""
            INSERT INTO DIM_USER_API (
                ID_USER_API_Source,
                Username,
                Email,
                Full_Name,
                Date_Creation,
                Date_Derniere_Connexion,
                Date_Debut_Validite,
                Date_Fin_Validite,
                Flag_Actuel
            )
            VALUES (?, ?, ?, ?, ?, ?, GETDATE(), NULL, 1)
            """, id_source, username, email, full_name, date_creation, date_derniere_connexion)
            update_count += 1
        else:
            skip_count += 1

cnxn_dwh.commit()

print(f"✅ Chargement terminé :")
print(f"   • {insert_count} nouveaux utilisateurs insérés")
print(f"   • {update_count} utilisateurs mis à jour (historisation)")
print(f"   • {skip_count} utilisateurs inchangés")
print("=" * 70)


📥 ETL - CHARGEMENT DE DIM_USER_API
⏳ Extraction des données USER_API depuis carter_cash...
✅ 4 utilisateurs extraits

⏳ Chargement dans DIM_USER_API avec gestion SCD Type 2...
✅ Chargement terminé :
   • 0 nouveaux utilisateurs insérés
   • 0 utilisateurs mis à jour (historisation)
   • 4 utilisateurs inchangés


Cellule 4 : ETL - Chargement de DIM_VEHICULE


In [11]:
from tqdm import tqdm

print("\n📥 ETL - CHARGEMENT DE DIM_VEHICULE (VERSION OPTIMISÉE)")
print("=" * 70)

# Extraction depuis la source
print("⏳ Extraction des données DimensionsParModel depuis carter_cash...")
cursor_source.execute("""
SELECT DISTINCT
    marque,
    modele,
    annee,
    finition,
    largeur,
    hauteur,
    diametre
FROM DimensionsParModel
WHERE marque IS NOT NULL AND modele IS NOT NULL
""")

vehicules_source = cursor_source.fetchall()
total_vehicules = len(vehicules_source)
print(f"✅ {total_vehicules} véhicules uniques extraits")

# ========================================
# OPTIMISATION : Charger les véhicules existants en mémoire
# ========================================
print("\n⏳ Chargement des véhicules existants en mémoire...")
cursor_dwh.execute("""
SELECT Marque, Modele, Annee, Finition
FROM DIM_VEHICULE
""")

vehicules_existants = set()
for row in cursor_dwh.fetchall():
    # Créer une clé unique pour chaque véhicule
    cle = (row[0], row[1], row[2], row[3])
    vehicules_existants.add(cle)

print(f"✅ {len(vehicules_existants)} véhicules déjà présents dans le DWH")

# ========================================
# Filtrer les véhicules à insérer
# ========================================
print("\n⏳ Filtrage des véhicules à insérer...")
vehicules_a_inserer = []

for vehicule in tqdm(vehicules_source, desc="Filtrage", unit="véhicule"):
    marque = vehicule[0]
    modele = vehicule[1]
    annee = vehicule[2]
    finition = vehicule[3]
    largeur = vehicule[4]
    hauteur = vehicule[5]
    diametre = vehicule[6]
    
    # Vérifier si le véhicule existe déjà
    cle = (marque, modele, annee, finition)
    
    if cle not in vehicules_existants:
        vehicules_a_inserer.append((marque, modele, annee, finition, largeur, hauteur, diametre))

skip_count = total_vehicules - len(vehicules_a_inserer)
print(f"✅ {len(vehicules_a_inserer)} nouveaux véhicules à insérer")
print(f"⚠️  {skip_count} véhicules déjà existants (ignorés)")

# ========================================
# Insertion par batch (plus rapide)
# ========================================
if len(vehicules_a_inserer) > 0:
    print("\n⏳ Insertion des nouveaux véhicules...")
    
    insert_query = """
    INSERT INTO DIM_VEHICULE (Marque, Modele, Annee, Finition, Largeur, Hauteur, Diametre)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """
    
    # Insertion par batch de 1000 lignes
    batch_size = 1000
    insert_count = 0
    
    for i in tqdm(range(0, len(vehicules_a_inserer), batch_size), desc="Insertion par batch", unit="batch"):
        batch = vehicules_a_inserer[i:i+batch_size]
        
        try:
            cursor_dwh.executemany(insert_query, batch)
            cnxn_dwh.commit()
            insert_count += len(batch)
        except Exception as e:
            print(f"\n⚠️  Erreur lors de l'insertion du batch {i//batch_size + 1} : {e}")
            # En cas d'erreur, insérer ligne par ligne pour ce batch
            for vehicule in batch:
                try:
                    cursor_dwh.execute(insert_query, vehicule)
                    insert_count += 1
                except:
                    pass
            cnxn_dwh.commit()
    
    print(f"\n✅ Insertion terminée : {insert_count} véhicules insérés")
else:
    print("\n✅ Aucun nouveau véhicule à insérer")
    insert_count = 0

# ========================================
# NETTOYAGE DES DOUBLONS
# ========================================
print("\n🧹 NETTOYAGE DES DOUBLONS")
print("=" * 70)

print("⏳ Détection des doublons...")
cursor_dwh.execute("""
SELECT Marque, Modele, Annee, Finition, COUNT(*) as Nb_Doublons
FROM DIM_VEHICULE
GROUP BY Marque, Modele, Annee, Finition
HAVING COUNT(*) > 1
""")

doublons = cursor_dwh.fetchall()
nb_groupes_doublons = len(doublons)

if nb_groupes_doublons > 0:
    print(f"⚠️  {nb_groupes_doublons} groupes de doublons trouvés")
    
    total_doublons_supprimes = 0
    
    # Supprimer les doublons pour chaque groupe
    for doublon in tqdm(doublons, desc="Suppression doublons", unit="groupe"):
        marque = doublon[0]
        modele = doublon[1]
        annee = doublon[2]
        finition = doublon[3]
        nb_occurrences = doublon[4]
        
        # Récupérer tous les SK_Vehicule pour ce groupe
        cursor_dwh.execute("""
        SELECT SK_Vehicule
        FROM DIM_VEHICULE
        WHERE Marque = ? AND Modele = ? AND Annee = ? AND Finition = ?
        ORDER BY SK_Vehicule ASC
        """, marque, modele, annee, finition)
        
        sk_vehicules = [row[0] for row in cursor_dwh.fetchall()]
        
        # Garder le premier (plus ancien), supprimer les autres
        sk_a_garder = sk_vehicules[0]
        sk_a_supprimer = sk_vehicules[1:]
        
        # Supprimer les doublons
        for sk in sk_a_supprimer:
            cursor_dwh.execute("""
            DELETE FROM DIM_VEHICULE
            WHERE SK_Vehicule = ?
            """, sk)
            total_doublons_supprimes += 1
    
    cnxn_dwh.commit()
    
    print(f"\n✅ Nettoyage terminé :")
    print(f"   • {nb_groupes_doublons} groupes de doublons traités")
    print(f"   • {total_doublons_supprimes} enregistrements en doublon supprimés")
else:
    print("✅ Aucun doublon trouvé dans DIM_VEHICULE")

# ========================================
# Vérification finale
# ========================================
print("\n🔍 VÉRIFICATION FINALE")
print("=" * 70)

cursor_dwh.execute("SELECT COUNT(*) FROM DIM_VEHICULE")
total_final = cursor_dwh.fetchone()[0]
print(f"📊 Nombre total de véhicules dans DIM_VEHICULE : {total_final}")

# Vérifier qu'il n'y a plus de doublons
cursor_dwh.execute("""
SELECT COUNT(*) 
FROM (
    SELECT Marque, Modele, Annee, Finition, COUNT(*) as Nb
    FROM DIM_VEHICULE
    GROUP BY Marque, Modele, Annee, Finition
    HAVING COUNT(*) > 1
) AS Doublons
""")
nb_doublons_restants = cursor_dwh.fetchone()[0]

if nb_doublons_restants == 0:
    print("✅ Aucun doublon détecté - Table propre !")
else:
    print(f"⚠️  ATTENTION : {nb_doublons_restants} doublons restants détectés")

print("\n" + "=" * 70)
print(f"✅ CHARGEMENT TERMINÉ :")
print(f"   • {insert_count} nouveaux véhicules insérés")
print(f"   • {skip_count} véhicules déjà existants (ignorés)")
if nb_groupes_doublons > 0:
    print(f"   • {total_doublons_supprimes} doublons supprimés")
print(f"   • {total_final} véhicules uniques au total")
print("=" * 70)


📥 ETL - CHARGEMENT DE DIM_VEHICULE (VERSION OPTIMISÉE)
⏳ Extraction des données DimensionsParModel depuis carter_cash...
✅ 187516 véhicules uniques extraits

⏳ Chargement des véhicules existants en mémoire...
✅ 51322 véhicules déjà présents dans le DWH

⏳ Filtrage des véhicules à insérer...


Filtrage: 100%|██████████| 187516/187516 [00:00<00:00, 562468.88véhicule/s]


✅ 136117 nouveaux véhicules à insérer
⚠️  51399 véhicules déjà existants (ignorés)

⏳ Insertion des nouveaux véhicules...


Insertion par batch:   7%|▋         | 9/137 [02:22<33:50, 15.86s/batch]


KeyboardInterrupt: 

Cellule 5 : ETL - Chargement de DIM_PRODUIT_PNEU


In [13]:
from tqdm import tqdm

print("\n📥 ETL - CHARGEMENT DE DIM_PRODUIT_PNEU")
print("=" * 70)

# Extraction depuis la source
print("⏳ Extraction des données PRODUIT depuis carter_cash...")
cursor_source.execute("""
SELECT 
    ID_Produit,
    URL_Produit,
    Descriptif,
    Marque,
    Info_generale,
    Note
FROM Produit
""")

produits_source = cursor_source.fetchall()
total_produits = len(produits_source)
print(f"✅ {total_produits} produits extraits")

# Chargement avec SCD Type 2
print("\n⏳ Chargement dans DIM_PRODUIT_PNEU avec gestion SCD Type 2...")

insert_count = 0
update_count = 0
skip_count = 0

# Barre de progression avec tqdm
for idx, produit in enumerate(tqdm(produits_source, desc="Chargement produits", unit="produit")):
    id_source = produit[0]
    url = produit[1]
    descriptif = produit[2]
    marque = produit[3]
    info_generale = produit[4]
    note = produit[5]
    
    # Vérifier si le produit existe déjà (version actuelle)
    cursor_dwh.execute("""
    SELECT SK_Produit, URL_Produit, Descriptif, Marque, Info_Generale, Note
    FROM DIM_PRODUIT_PNEU
    WHERE ID_Produit_Source = ? AND Flag_Actuel = 1
    """, id_source)
    
    existing_produit = cursor_dwh.fetchone()
    
    if existing_produit is None:
        # Nouveau produit : insertion
        cursor_dwh.execute("""
        INSERT INTO DIM_PRODUIT_PNEU (
            ID_Produit_Source,
            URL_Produit,
            Descriptif,
            Marque,
            Info_Generale,
            Note,
            Date_Debut_Validite,
            Date_Fin_Validite,
            Flag_Actuel
        )
        VALUES (?, ?, ?, ?, ?, ?, GETDATE(), NULL, 1)
        """, id_source, url, descriptif, marque, info_generale, note)
        insert_count += 1
        
    else:
        # Vérifier si les données ont changé
        sk_produit = existing_produit[0]
        old_url = existing_produit[1]
        old_descriptif = existing_produit[2]
        old_marque = existing_produit[3]
        old_info = existing_produit[4]
        old_note = existing_produit[5]
        
        if (url != old_url or descriptif != old_descriptif or 
            marque != old_marque or info_generale != old_info or note != old_note):
            
            # Fermer l'ancienne version
            cursor_dwh.execute("""
            UPDATE DIM_PRODUIT_PNEU
            SET Date_Fin_Validite = GETDATE(), Flag_Actuel = 0
            WHERE SK_Produit = ?
            """, sk_produit)
            
            # Insérer la nouvelle version
            cursor_dwh.execute("""
            INSERT INTO DIM_PRODUIT_PNEU (
                ID_Produit_Source,
                URL_Produit,
                Descriptif,
                Marque,
                Info_Generale,
                Note,
                Date_Debut_Validite,
                Date_Fin_Validite,
                Flag_Actuel
            )
            VALUES (?, ?, ?, ?, ?, ?, GETDATE(), NULL, 1)
            """, id_source, url, descriptif, marque, info_generale, note)
            update_count += 1
        else:
            skip_count += 1
    
    # Commit tous les 100 enregistrements pour éviter les timeouts
    if (idx + 1) % 100 == 0:
        cnxn_dwh.commit()

# Commit final
cnxn_dwh.commit()

print(f"\n✅ Chargement terminé :")
print(f"   • {insert_count} nouveaux produits insérés")
print(f"   • {update_count} produits mis à jour (historisation)")
print(f"   • {skip_count} produits inchangés")
print("=" * 70)


📥 ETL - CHARGEMENT DE DIM_PRODUIT_PNEU
⏳ Extraction des données PRODUIT depuis carter_cash...
✅ 11442 produits extraits

⏳ Chargement dans DIM_PRODUIT_PNEU avec gestion SCD Type 2...


Chargement produits: 100%|██████████| 11442/11442 [06:44<00:00, 28.31produit/s]


✅ Chargement terminé :
   • 11442 nouveaux produits insérés
   • 0 produits mis à jour (historisation)
   • 0 produits inchangés


Cellule 6 : ETL - Chargement de DIM_CARACTERISTIQUES_PNEU


In [14]:
from tqdm import tqdm

print("\n📥 ETL - CHARGEMENT DE DIM_CARACTERISTIQUES_PNEU")
print("=" * 70)

# Extraction depuis la source
print("⏳ Extraction des données CARACTERISTIQUES depuis carter_cash...")
cursor_source.execute("""
SELECT DISTINCT
    Saisonalite,
    Type_Vehicule,
    Consommation,
    Indice_Pluie,
    Bruit,
    Runflat
FROM Caracteristiques
""")

caracteristiques_source = cursor_source.fetchall()
total_caracteristiques = len(caracteristiques_source)
print(f"✅ {total_caracteristiques} caractéristiques uniques extraites")

# Chargement dans DIM_CARACTERISTIQUES_PNEU
print("\n⏳ Chargement dans DIM_CARACTERISTIQUES_PNEU...")

insert_count = 0
skip_count = 0

# Barre de progression avec tqdm
for idx, carac in enumerate(tqdm(caracteristiques_source, desc="Chargement caractéristiques", unit="carac")):
    saisonalite = carac[0]
    type_vehicule = carac[1]
    consommation = carac[2]
    indice_pluie = carac[3]
    bruit = carac[4]
    runflat = carac[5]
    
    # Vérifier si la caractéristique existe déjà
    cursor_dwh.execute("""
    SELECT SK_Caracteristique
    FROM DIM_CARACTERISTIQUES_PNEU
    WHERE Saisonalite = ? AND Type_Vehicule = ? AND Consommation = ? 
    AND Indice_Pluie = ? AND Bruit = ? AND Runflat = ?
    """, saisonalite, type_vehicule, consommation, indice_pluie, bruit, runflat)
    
    existing = cursor_dwh.fetchone()
    
    if existing is None:
        # Insertion de la nouvelle caractéristique
        cursor_dwh.execute("""
        INSERT INTO DIM_CARACTERISTIQUES_PNEU (
            Saisonalite,
            Type_Vehicule,
            Consommation,
            Indice_Pluie,
            Bruit,
            Runflat
        )
        VALUES (?, ?, ?, ?, ?, ?)
        """, saisonalite, type_vehicule, consommation, indice_pluie, bruit, runflat)
        insert_count += 1
    else:
        skip_count += 1
    
    # Commit tous les 50 enregistrements pour éviter les timeouts
    if (idx + 1) % 50 == 0:
        cnxn_dwh.commit()

# Commit final
cnxn_dwh.commit()

print(f"\n✅ Chargement terminé :")
print(f"   • {insert_count} nouvelles caractéristiques insérées")
print(f"   • {skip_count} caractéristiques déjà existantes")
print("=" * 70)


📥 ETL - CHARGEMENT DE DIM_CARACTERISTIQUES_PNEU
⏳ Extraction des données CARACTERISTIQUES depuis carter_cash...
✅ 687 caractéristiques uniques extraites

⏳ Chargement dans DIM_CARACTERISTIQUES_PNEU...


Chargement caractéristiques: 100%|██████████| 687/687 [00:21<00:00, 32.18carac/s]


✅ Chargement terminé :
   • 687 nouvelles caractéristiques insérées
   • 0 caractéristiques déjà existantes


Cellule 7 : ETL - Chargement de DIM_DIMENSIONS_PNEU


In [16]:
from tqdm import tqdm

print("\n📥 ETL - CHARGEMENT DE DIM_DIMENSIONS_PNEU")
print("=" * 70)

# Extraction depuis la source
print("⏳ Extraction des données DIMENSIONS depuis carter_cash...")
cursor_source.execute("""
SELECT DISTINCT
    Largeur,
    Hauteur,
    Diametre,
    Charge,
    Vitesse
FROM Dimensions
WHERE Largeur IS NOT NULL AND Hauteur IS NOT NULL AND Diametre IS NOT NULL
""")

dimensions_source = cursor_source.fetchall()
total_dimensions = len(dimensions_source)
print(f"✅ {total_dimensions} dimensions uniques extraites")

# Chargement dans DIM_DIMENSIONS_PNEU
print("\n⏳ Chargement dans DIM_DIMENSIONS_PNEU...")

insert_count = 0
skip_count = 0

# Barre de progression avec tqdm
for idx, dim in enumerate(tqdm(dimensions_source, desc="Chargement dimensions", unit="dimension")):
    largeur = dim[0]
    hauteur = dim[1]
    diametre = dim[2]
    charge = dim[3]
    vitesse = dim[4]
    
    # Créer la taille standard au format "Largeur/Hauteur R Diametre"
    taille_standard = f"{largeur}/{hauteur} R{diametre}" if largeur and hauteur and diametre else None
    
    # Vérifier si la dimension existe déjà
    cursor_dwh.execute("""
    SELECT SK_Dimension_Pneu
    FROM DIM_DIMENSIONS_PNEU
    WHERE Largeur = ? AND Hauteur = ? AND Diametre = ? AND Charge = ? AND Vitesse = ?
    """, largeur, hauteur, diametre, charge, vitesse)
    
    existing = cursor_dwh.fetchone()
    
    if existing is None:
        # Insertion de la nouvelle dimension
        cursor_dwh.execute("""
        INSERT INTO DIM_DIMENSIONS_PNEU (
            Largeur,
            Hauteur,
            Diametre,
            Charge,
            Vitesse,
            Taille_Standard
        )
        VALUES (?, ?, ?, ?, ?, ?)
        """, largeur, hauteur, diametre, charge, vitesse, taille_standard)
        insert_count += 1
    else:
        skip_count += 1
    
    # Commit tous les 50 enregistrements pour éviter les timeouts
    if (idx + 1) % 50 == 0:
        cnxn_dwh.commit()

# Commit final
cnxn_dwh.commit()

print(f"\n✅ Chargement terminé :")
print(f"   • {insert_count} nouvelles dimensions insérées")
print(f"   • {skip_count} dimensions déjà existantes")
print("=" * 70)


📥 ETL - CHARGEMENT DE DIM_DIMENSIONS_PNEU
⏳ Extraction des données DIMENSIONS depuis carter_cash...
✅ 747 dimensions uniques extraites

⏳ Chargement dans DIM_DIMENSIONS_PNEU...


Chargement dimensions: 100%|██████████| 747/747 [00:09<00:00, 80.73dimension/s]


✅ Chargement terminé :
   • 0 nouvelles dimensions insérées
   • 747 dimensions déjà existantes


Cellule 8 : ETL - Chargement de FAIT_PRIX_PNEU


In [17]:
from tqdm import tqdm

print("\n📥 ETL - CHARGEMENT DE FAIT_PRIX_PNEU")
print("=" * 70)

# Extraction depuis la source avec jointures
print("⏳ Extraction des données avec jointures depuis carter_cash...")
cursor_source.execute("""
SELECT 
    p.ID_Produit,
    p.Prix,
    p.Date_scrap,
    c.Consommation,
    c.Indice_Pluie,
    c.Bruit,
    c.Saisonalite,
    c.Type_Vehicule,
    c.Runflat,
    d.Largeur,
    d.Hauteur,
    d.Diametre,
    d.Charge,
    d.Vitesse
FROM Produit p
INNER JOIN Caracteristiques c ON p.ID_Produit = c.ID_Produit
INNER JOIN Dimensions d ON p.ID_Produit = d.ID_Produit
WHERE p.Prix IS NOT NULL AND p.Date_scrap IS NOT NULL
""")

faits_source = cursor_source.fetchall()
total_faits = len(faits_source)
print(f"✅ {total_faits} enregistrements extraits")

# Chargement dans FAIT_PRIX_PNEU
print("\n⏳ Chargement dans FAIT_PRIX_PNEU...")

insert_count = 0
skip_count = 0
error_count = 0
error_details = {
    'produit_manquant': 0,
    'caracteristique_manquante': 0,
    'dimension_manquante': 0,
    'temps_manquant': 0,
    'autre': 0
}

# Barre de progression avec tqdm
for idx, fait in enumerate(tqdm(faits_source, desc="Chargement faits prix", unit="fait")):
    try:
        id_produit_source = fait[0]
        prix = fait[1]
        date_scrap = fait[2]
        consommation = fait[3]
        indice_pluie = fait[4]
        bruit = fait[5]
        saisonalite = fait[6]
        type_vehicule = fait[7]
        runflat = fait[8]
        largeur = fait[9]
        hauteur = fait[10]
        diametre = fait[11]
        charge = fait[12]
        vitesse = fait[13]
        
        # Récupérer SK_Produit depuis DIM_PRODUIT_PNEU
        cursor_dwh.execute("""
        SELECT SK_Produit
        FROM DIM_PRODUIT_PNEU
        WHERE ID_Produit_Source = ? AND Flag_Actuel = 1
        """, id_produit_source)
        
        sk_produit_row = cursor_dwh.fetchone()
        if sk_produit_row is None:
            error_count += 1
            error_details['produit_manquant'] += 1
            continue
        sk_produit = sk_produit_row[0]
        
        # Récupérer SK_Caracteristique depuis DIM_CARACTERISTIQUES_PNEU
        cursor_dwh.execute("""
        SELECT SK_Caracteristique
        FROM DIM_CARACTERISTIQUES_PNEU
        WHERE Saisonalite = ? AND Type_Vehicule = ? AND Consommation = ? 
        AND Indice_Pluie = ? AND Bruit = ? AND Runflat = ?
        """, saisonalite, type_vehicule, consommation, indice_pluie, bruit, runflat)
        
        sk_carac_row = cursor_dwh.fetchone()
        if sk_carac_row is None:
            error_count += 1
            error_details['caracteristique_manquante'] += 1
            continue
        sk_caracteristique = sk_carac_row[0]
        
        # Récupérer SK_Dimension_Pneu depuis DIM_DIMENSIONS_PNEU
        cursor_dwh.execute("""
        SELECT SK_Dimension_Pneu
        FROM DIM_DIMENSIONS_PNEU
        WHERE Largeur = ? AND Hauteur = ? AND Diametre = ? AND Charge = ? AND Vitesse = ?
        """, largeur, hauteur, diametre, charge, vitesse)
        
        sk_dim_row = cursor_dwh.fetchone()
        if sk_dim_row is None:
            error_count += 1
            error_details['dimension_manquante'] += 1
            continue
        sk_dimension = sk_dim_row[0]
        
        # Récupérer SK_Temps depuis DIM_TEMPS
        cursor_dwh.execute("""
        SELECT SK_Temps
        FROM DIM_TEMPS
        WHERE Date = ?
        """, date_scrap)
        
        sk_temps_row = cursor_dwh.fetchone()
        if sk_temps_row is None:
            error_count += 1
            error_details['temps_manquant'] += 1
            continue
        sk_temps = sk_temps_row[0]
        
        # Vérifier si l'enregistrement existe déjà dans FAIT_PRIX_PNEU
        cursor_dwh.execute("""
        SELECT 1
        FROM FAIT_PRIX_PNEU
        WHERE SK_Temps = ? AND SK_Produit = ? AND SK_Caracteristique = ? 
        AND SK_Dimension_Pneu = ? AND Date_Scrap = ?
        """, sk_temps, sk_produit, sk_caracteristique, sk_dimension, date_scrap)
        
        existing = cursor_dwh.fetchone()
        
        if existing is None:
            # Insérer dans FAIT_PRIX_PNEU
            cursor_dwh.execute("""
            INSERT INTO FAIT_PRIX_PNEU (
                SK_Temps,
                SK_Produit,
                SK_Caracteristique,
                SK_Dimension_Pneu,
                Prix_Euro,
                Date_Scrap
            )
            VALUES (?, ?, ?, ?, ?, ?)
            """, sk_temps, sk_produit, sk_caracteristique, sk_dimension, prix, date_scrap)
            insert_count += 1
        else:
            skip_count += 1
        
        # Commit tous les 100 enregistrements pour éviter les timeouts
        if (idx + 1) % 100 == 0:
            cnxn_dwh.commit()
            
    except Exception as e:
        error_count += 1
        error_details['autre'] += 1

# Commit final
cnxn_dwh.commit()

print(f"\n✅ Chargement terminé :")
print(f"   • {insert_count} nouveaux faits insérés")
print(f"   • {skip_count} faits déjà existants")
print(f"   • {error_count} erreurs (clés manquantes)")

if error_count > 0:
    print(f"\n⚠️  Détail des erreurs :")
    if error_details['produit_manquant'] > 0:
        print(f"   • Produits non trouvés : {error_details['produit_manquant']}")
    if error_details['caracteristique_manquante'] > 0:
        print(f"   • Caractéristiques non trouvées : {error_details['caracteristique_manquante']}")
    if error_details['dimension_manquante'] > 0:
        print(f"   • Dimensions non trouvées : {error_details['dimension_manquante']}")
    if error_details['temps_manquant'] > 0:
        print(f"   • Dates non trouvées dans DIM_TEMPS : {error_details['temps_manquant']}")
    if error_details['autre'] > 0:
        print(f"   • Autres erreurs : {error_details['autre']}")

print("=" * 70)


📥 ETL - CHARGEMENT DE FAIT_PRIX_PNEU
⏳ Extraction des données avec jointures depuis carter_cash...
✅ 11180 enregistrements extraits

⏳ Chargement dans FAIT_PRIX_PNEU...


Chargement faits prix: 100%|██████████| 11180/11180 [14:20<00:00, 12.99fait/s]


✅ Chargement terminé :
   • 11180 nouveaux faits insérés
   • 0 faits déjà existants
   • 0 erreurs (clés manquantes)


Cellule 9 : Vérification finale des données chargées


In [18]:
print("\n🔍 VÉRIFICATION FINALE DES DONNÉES CHARGÉES")
print("=" * 70)

# Vérifier DIM_USER_API
cursor_dwh.execute("SELECT COUNT(*) FROM DIM_USER_API WHERE Flag_Actuel = 1")
count_users = cursor_dwh.fetchone()[0]
print(f"✅ DIM_USER_API : {count_users} utilisateurs actifs")

# Vérifier DIM_VEHICULE
cursor_dwh.execute("SELECT COUNT(*) FROM DIM_VEHICULE")
count_vehicules = cursor_dwh.fetchone()[0]
print(f"✅ DIM_VEHICULE : {count_vehicules} véhicules")

# Vérifier DIM_PRODUIT_PNEU
cursor_dwh.execute("SELECT COUNT(*) FROM DIM_PRODUIT_PNEU WHERE Flag_Actuel = 1")
count_produits = cursor_dwh.fetchone()[0]
print(f"✅ DIM_PRODUIT_PNEU : {count_produits} produits actifs")

# Vérifier DIM_CARACTERISTIQUES_PNEU
cursor_dwh.execute("SELECT COUNT(*) FROM DIM_CARACTERISTIQUES_PNEU")
count_carac = cursor_dwh.fetchone()[0]
print(f"✅ DIM_CARACTERISTIQUES_PNEU : {count_carac} caractéristiques")

# Vérifier DIM_DIMENSIONS_PNEU
cursor_dwh.execute("SELECT COUNT(*) FROM DIM_DIMENSIONS_PNEU")
count_dim = cursor_dwh.fetchone()[0]
print(f"✅ DIM_DIMENSIONS_PNEU : {count_dim} dimensions")

# Vérifier FAIT_PRIX_PNEU
cursor_dwh.execute("SELECT COUNT(*) FROM FAIT_PRIX_PNEU")
count_faits = cursor_dwh.fetchone()[0]
print(f"✅ FAIT_PRIX_PNEU : {count_faits} enregistrements")

print("\n" + "=" * 70)

# Statistiques détaillées sur FAIT_PRIX_PNEU
print("\n📊 STATISTIQUES DÉTAILLÉES SUR FAIT_PRIX_PNEU :")
print("=" * 70)

# Prix moyen
cursor_dwh.execute("SELECT AVG(Prix_Euro) FROM FAIT_PRIX_PNEU")
prix_moyen = cursor_dwh.fetchone()[0]
print(f"💰 Prix moyen des pneus : {prix_moyen:.2f} €" if prix_moyen else "💰 Prix moyen : N/A")

# Prix min et max
cursor_dwh.execute("SELECT MIN(Prix_Euro), MAX(Prix_Euro) FROM FAIT_PRIX_PNEU")
prix_minmax = cursor_dwh.fetchone()
if prix_minmax[0] and prix_minmax[1]:
    print(f"📉 Prix minimum : {prix_minmax[0]:.2f} €")
    print(f"📈 Prix maximum : {prix_minmax[1]:.2f} €")

# Répartition par date de scraping
cursor_dwh.execute("""
SELECT TOP 5 Date_Scrap, COUNT(*) as Nb_Enregistrements
FROM FAIT_PRIX_PNEU
GROUP BY Date_Scrap
ORDER BY Date_Scrap DESC
""")

print("\n📅 Dernières dates de scraping :")
for row in cursor_dwh.fetchall():
    print(f"   • {row[0]} : {row[1]} enregistrements")

print("=" * 70)


🔍 VÉRIFICATION FINALE DES DONNÉES CHARGÉES
✅ DIM_USER_API : 4 utilisateurs actifs
✅ DIM_VEHICULE : 63322 véhicules
✅ DIM_PRODUIT_PNEU : 11442 produits actifs
✅ DIM_CARACTERISTIQUES_PNEU : 687 caractéristiques
✅ DIM_DIMENSIONS_PNEU : 747 dimensions
✅ FAIT_PRIX_PNEU : 11180 enregistrements


📊 STATISTIQUES DÉTAILLÉES SUR FAIT_PRIX_PNEU :
💰 Prix moyen des pneus : 114.30 €
📉 Prix minimum : 35.00 €
📈 Prix maximum : 399.00 €

📅 Dernières dates de scraping :
   • 2025-04-20 : 7177 enregistrements
   • 2024-08-02 : 4003 enregistrements


Cellule 10 : Analyse de qualité des données (COMPLÈTE)


In [19]:
print("\n🔬 ANALYSE DE QUALITÉ DES DONNÉES")
print("=" * 70)

# Vérifier les produits sans prix
cursor_dwh.execute("""
SELECT COUNT(DISTINCT SK_Produit)
FROM DIM_PRODUIT_PNEU
WHERE Flag_Actuel = 1
AND SK_Produit NOT IN (SELECT DISTINCT SK_Produit FROM FAIT_PRIX_PNEU)
""")
produits_sans_prix = cursor_dwh.fetchone()[0]
print(f"⚠️  Produits sans prix dans les faits : {produits_sans_prix}")

# Top 5 des marques les plus présentes
cursor_dwh.execute("""
SELECT TOP 5 p.Marque, COUNT(*) as Nb_Produits
FROM DIM_PRODUIT_PNEU p
WHERE p.Flag_Actuel = 1
GROUP BY p.Marque
ORDER BY Nb_Produits DESC
""")

print("\n🏆 Top 5 des marques de pneus :")
for row in cursor_dwh.fetchall():
    print(f"   • {row[0]} : {row[1]} produits")

# Répartition par saisonnalité
cursor_dwh.execute("""
SELECT c.Saisonalite, COUNT(*) as Nb_Produits
FROM FAIT_PRIX_PNEU f
INNER JOIN DIM_CARACTERISTIQUES_PNEU c ON f.SK_Caracteristique = c.SK_Caracteristique
GROUP BY c.Saisonalite
ORDER BY Nb_Produits DESC
""")

print("\n🌦️  Répartition par saisonnalité :")
for row in cursor_dwh.fetchall():
    print(f"   • {row[0]} : {row[1]} enregistrements")

# Répartition par type de véhicule
cursor_dwh.execute("""
SELECT c.Type_Vehicule, COUNT(*) as Nb_Produits
FROM FAIT_PRIX_PNEU f
INNER JOIN DIM_CARACTERISTIQUES_PNEU c ON f.SK_Caracteristique = c.SK_Caracteristique
GROUP BY c.Type_Vehicule
ORDER BY Nb_Produits DESC
""")

print("\n🚗 Répartition par type de véhicule :")
for row in cursor_dwh.fetchall():
    print(f"   • {row[0]} : {row[1]} enregistrements")

# Top 5 des dimensions les plus courantes
cursor_dwh.execute("""
SELECT TOP 5 d.Taille_Standard, COUNT(*) as Nb_Enregistrements
FROM FAIT_PRIX_PNEU f
INNER JOIN DIM_DIMENSIONS_PNEU d ON f.SK_Dimension_Pneu = d.SK_Dimension_Pneu
WHERE d.Taille_Standard IS NOT NULL
GROUP BY d.Taille_Standard
ORDER BY Nb_Enregistrements DESC
""")

print("\n📏 Top 5 des tailles de pneus :")
for row in cursor_dwh.fetchall():
    print(f"   • {row[0]} : {row[1]} enregistrements")

# Top 5 des marques de véhicules
cursor_dwh.execute("""
SELECT TOP 5 Marque, COUNT(*) as Nb_Vehicules
FROM DIM_VEHICULE
GROUP BY Marque
ORDER BY Nb_Vehicules DESC
""")

print("\n🏭 Top 5 des marques de véhicules :")
for row in cursor_dwh.fetchall():
    print(f"   • {row[0]} : {row[1]} véhicules")

print("=" * 70)


🔬 ANALYSE DE QUALITÉ DES DONNÉES
⚠️  Produits sans prix dans les faits : 262

🏆 Top 5 des marques de pneus :
   • None : 7345 produits
   • MICHELIN : 657 produits
   • CONTINENTAL : 591 produits
   • HANKOOK : 505 produits
   • GOODYEAR : 345 produits

🌦️  Répartition par saisonnalité :
   • Été : 6838 enregistrements
   • 4 saisons : 2649 enregistrements
   • Hiver : 1693 enregistrements

🚗 Répartition par type de véhicule :
   • Tourisme : 8704 enregistrements
   • 4x4 : 1491 enregistrements
   • Utilitaire : 985 enregistrements

📏 Top 5 des tailles de pneus :
   • 205/55 R16 : 406 enregistrements
   • 195/65 R15 : 282 enregistrements
   • 225/45 R17 : 278 enregistrements
   • 205/60 R16 : 262 enregistrements
   • 195/55 R16 : 242 enregistrements

🏭 Top 5 des marques de véhicules :
   • mercedes : 7433 véhicules
   • renault : 6699 véhicules
   • volkswagen : 6666 véhicules
   • peugeot : 4289 véhicules
   • volvo : 3434 véhicules


Cellule 11 : Statistiques temporelles


In [20]:
# Fermeture des connexions
cursor_source.close()
cnxn_source.close()
cursor_dwh.close()
cnxn_dwh.close()

print("\n" + "=" * 70)
print("✅ CONNEXIONS FERMÉES")
print("🎉 ETL TERMINÉ AVEC SUCCÈS !")
print("=" * 70)

print("\n📊 RÉCAPITULATIF DE L'ETL :")
print("\n✅ Tables alimentées :")
print(f"   1. DIM_USER_API : Utilisateurs chargés avec SCD Type 2")
print(f"   2. DIM_VEHICULE : Véhicules chargés")
print(f"   3. DIM_PRODUIT_PNEU : Produits pneus chargés avec SCD Type 2")
print(f"   4. DIM_CARACTERISTIQUES_PNEU : Caractéristiques chargées")
print(f"   5. DIM_DIMENSIONS_PNEU : Dimensions chargées")
print(f"   6. FAIT_PRIX_PNEU : Faits prix chargés")

print("\n🎯 PROCHAINES ÉTAPES :")
print("   1️⃣  Charger les données des bornes de recharge (DIM_BORNE_RECHARGE)")
print("   2️⃣  Charger les données géographiques (DIM_GEOGRAPHIE)")
print("   3️⃣  Charger les données entreprises (DIM_ENTREPRISE, DIM_ACTIVITE_NAF)")
print("   4️⃣  Alimenter les tables de faits restantes")
print("   5️⃣  Créer les vues pour Power BI")

print("\n💡 Le Data Warehouse est maintenant alimenté avec les données Carter Cash !")
print("=" * 70)


✅ CONNEXIONS FERMÉES
🎉 ETL TERMINÉ AVEC SUCCÈS !

📊 RÉCAPITULATIF DE L'ETL :

✅ Tables alimentées :
   1. DIM_USER_API : Utilisateurs chargés avec SCD Type 2
   2. DIM_VEHICULE : Véhicules chargés
   3. DIM_PRODUIT_PNEU : Produits pneus chargés avec SCD Type 2
   4. DIM_CARACTERISTIQUES_PNEU : Caractéristiques chargées
   5. DIM_DIMENSIONS_PNEU : Dimensions chargées
   6. FAIT_PRIX_PNEU : Faits prix chargés

🎯 PROCHAINES ÉTAPES :
   1️⃣  Charger les données des bornes de recharge (DIM_BORNE_RECHARGE)
   2️⃣  Charger les données géographiques (DIM_GEOGRAPHIE)
   3️⃣  Charger les données entreprises (DIM_ENTREPRISE, DIM_ACTIVITE_NAF)
   4️⃣  Alimenter les tables de faits restantes
   5️⃣  Créer les vues pour Power BI

💡 Le Data Warehouse est maintenant alimenté avec les données Carter Cash !


Cellule 12 : Fermeture des connexions et récapitulatif final


In [ ]:
# Fermeture des connexions
cursor_source.close()
cnxn_source.close()
cursor_dwh.close()
cnxn_dwh.close()

print("\n" + "=" * 70)
print("✅ CONNEXIONS FERMÉES")
print("🎉 ETL TERMINÉ AVEC SUCCÈS !")
print("=" * 70)

print("\n📊 RÉCAPITULATIF DE L'ETL :")
print("\n✅ Tables alimentées :")
print(f"   1. DIM_USER_API : Utilisateurs chargés avec SCD Type 2")
print(f"   2. DIM_VEHICULE : Véhicules chargés")
print(f"   3. DIM_PRODUIT_PNEU : Produits pneus chargés avec SCD Type 2")
print(f"   4. DIM_CARACTERISTIQUES_PNEU : Caractéristiques chargées")
print(f"   5. DIM_DIMENSIONS_PNEU : Dimensions chargées")
print(f"   6. FAIT_PRIX_PNEU : Faits prix chargés")

print("\n🎯 PROCHAINES ÉTAPES :")
print("   1️⃣  Charger les données des bornes de recharge (DIM_BORNE_RECHARGE)")
print("   2️⃣  Charger les données géographiques (DIM_GEOGRAPHIE)")
print("   3️⃣  Charger les données entreprises (DIM_ENTREPRISE, DIM_ACTIVITE_NAF)")
print("   4️⃣  Alimenter les tables de faits restantes")
print("   5️⃣  Créer les vues pour Power BI")

print("\n💡 Le Data Warehouse est maintenant alimenté avec les données Carter Cash !")
print("=" * 70)